# Time Dependent Evolution

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from timedependent_channels import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=5, suppress=True)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Interacting Hamiltonian

## Fit CNOT

In [2]:
T = 1
U = tf.convert_to_tensor([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 0, 1],
                            [0, 0, 1, 0]], dtype=tf.complex128)

channel_target = ChoiMapStatic(U, mode="unitary")
choi_target = channel_target.choi
n = 2
d = 2**n

tf.random.set_seed(42)
np.random.seed(42)

H_model = SpinSpin(degree=3)
jump_operator = JumpOperator(4, trainable=False)

lindblad_model = LindbladGenerator(hamiltonian = H_model, 
                                   jump_operator = jump_operator,
                                   gamma = 0.0)

channel_model = MagnusPropagator(liouvillian=lindblad_model, grid_size=200)
print(tf.math.reduce_sum(tf.abs(channel_model.choi(T) - choi_target)**2))

tf.Tensor(31.917988443199974, shape=(), dtype=float64)


In [3]:
T = 1

optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_model.parameter_list)
        choi_model = channel_model.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_model.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_model.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_model.parameter_list))

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(31.917988443199974, shape=(), dtype=float64)
tf.Tensor(31.879549092655644, shape=(), dtype=float64)
tf.Tensor(31.834301050787424, shape=(), dtype=float64)
tf.Tensor(31.78247478321377, shape=(), dtype=float64)
tf.Tensor(31.72418087545993, shape=(), dtype=float64)
tf.Tensor(31.65934394535158, shape=(), dtype=float64)
tf.Tensor(31.58760445467837, shape=(), dtype=float64)
tf.Tensor(31.50855332681749, shape=(), dtype=float64)
tf.Tensor(31.421947808342637, shape=(), dtype=float64)
tf.Tensor(31.327669217972332, shape=(), dtype=float64)
tf.Tensor(31.22568397824668, shape=(), dtype=float64)
tf.Tensor(31.116011555149186, shape=(), dtype=float64)
tf.Tensor(30.99868621363754, shape=(), dtype=float64)
tf.Tensor(30.873701335159183, shape=(), dtype=float64)
tf.Tensor(30.740934194721188, shape=(), dtype=float64)
tf.Tensor(30.600080769867116, shape=(), dtype=float64)
tf.Tensor(30.450675851147892, shape=(), dtype=float64)
tf.Tensor(30.292225742541603, shape=(), dtype=float64)
tf.Tensor(30.1243

In [9]:
channel_noisy = deepcopy(channel_model)
channel_noisy.liouvillian.gamma = 0.25
print(state_fidelity(choi_target, channel_model.choi(T))/16)
print(state_fidelity(choi_target, channel_noisy.choi(T))/16)

tf.Tensor(0.9999997615504477, shape=(), dtype=float64)
tf.Tensor(0.8752775050360572, shape=(), dtype=float64)


In [10]:
T = 1

optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_noisy.parameter_list)
        choi_model = channel_noisy.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_noisy.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_noisy.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_noisy.parameter_list))

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(0.3636235428971628, shape=(), dtype=float64)
tf.Tensor(0.35101286606225834, shape=(), dtype=float64)
tf.Tensor(0.3466769981880262, shape=(), dtype=float64)
tf.Tensor(0.3493872962090816, shape=(), dtype=float64)
tf.Tensor(0.34804427261521964, shape=(), dtype=float64)
tf.Tensor(0.3459567055187853, shape=(), dtype=float64)
tf.Tensor(0.34445829414425033, shape=(), dtype=float64)
tf.Tensor(0.34376733646384694, shape=(), dtype=float64)
tf.Tensor(0.3440365001298587, shape=(), dtype=float64)
tf.Tensor(0.34449728842183425, shape=(), dtype=float64)
tf.Tensor(0.3444454659470434, shape=(), dtype=float64)
tf.Tensor(0.34395551671974545, shape=(), dtype=float64)
tf.Tensor(0.3430493527501402, shape=(), dtype=float64)
tf.Tensor(0.3420321261270358, shape=(), dtype=float64)
tf.Tensor(0.34155168418346943, shape=(), dtype=float64)
tf.Tensor(0.34181931824368406, shape=(), dtype=float64)
tf.Tensor(0.3423742485759522, shape=(), dtype=float64)
tf.Tensor(0.3426204060981028, shape=(), dtype=float64)
tf

In [12]:
print(state_fidelity(choi_target, channel_model.choi(T))/16)
print(state_fidelity(choi_target, channel_noisy.choi(T))/16)

tf.Tensor(0.9999997615504477, shape=(), dtype=float64)
tf.Tensor(0.8758372386006208, shape=(), dtype=float64)


In [3]:
T = 1
U = tf.convert_to_tensor([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 0, 1],
                            [0, 0, 1, 0]], dtype=tf.complex128)

channel_target = ChoiMapStatic(U, mode="unitary")
choi_target = channel_target.choi
n = 2
d = 2**n

tf.random.set_seed(42)
np.random.seed(42)

H_model = SpinSpin(degree=3)
jump_operator = JumpOperator(4, trainable=True)

lindblad_model = LindbladGenerator(hamiltonian = H_model, 
                                   jump_operator = jump_operator,
                                   gamma = 0.0)

channel_model = MagnusPropagator(liouvillian=lindblad_model, grid_size=200)
print(tf.math.reduce_sum(tf.abs(channel_model.choi(T) - choi_target)**2))

tf.Tensor(31.917988443199974, shape=(), dtype=float64)


In [4]:
T = 1

optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_model.parameter_list)
        choi_model = channel_model.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_model.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_model.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_model.parameter_list))

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(31.917988443199974, shape=(), dtype=float64)
tf.Tensor(31.879549092655644, shape=(), dtype=float64)
tf.Tensor(31.834301050787424, shape=(), dtype=float64)
tf.Tensor(31.78247478321377, shape=(), dtype=float64)
tf.Tensor(31.72418087545993, shape=(), dtype=float64)
tf.Tensor(31.65934394535158, shape=(), dtype=float64)
tf.Tensor(31.58760445467837, shape=(), dtype=float64)
tf.Tensor(31.50855332681749, shape=(), dtype=float64)
tf.Tensor(31.421947808342637, shape=(), dtype=float64)
tf.Tensor(31.327669217972332, shape=(), dtype=float64)
tf.Tensor(31.22568397824668, shape=(), dtype=float64)
tf.Tensor(31.116011555149186, shape=(), dtype=float64)
tf.Tensor(30.99868621363754, shape=(), dtype=float64)
tf.Tensor(30.873701335159183, shape=(), dtype=float64)
tf.Tensor(30.740934194721188, shape=(), dtype=float64)
tf.Tensor(30.600080769867116, shape=(), dtype=float64)
tf.Tensor(30.450675851147892, shape=(), dtype=float64)
tf.Tensor(30.292225742541603, shape=(), dtype=float64)
tf.Tensor(30.1243

In [5]:
channel_noisy = deepcopy(channel_model)
channel_noisy.liouvillian.gamma = 0.25
print(state_fidelity(choi_target, channel_model.choi(T))/16)
print(state_fidelity(choi_target, channel_noisy.choi(T))/16)

tf.Tensor(0.9999997615504477, shape=(), dtype=float64)
tf.Tensor(0.9529010283503321, shape=(), dtype=float64)


In [6]:
T = 1

optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_noisy.parameter_list)
        choi_model = channel_noisy.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_noisy.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_noisy.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_noisy.parameter_list))

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(0.0530236425598718, shape=(), dtype=float64)
tf.Tensor(0.06127725035745801, shape=(), dtype=float64)
tf.Tensor(0.05520455783573885, shape=(), dtype=float64)
tf.Tensor(0.051041760422829346, shape=(), dtype=float64)
tf.Tensor(0.04777992773587475, shape=(), dtype=float64)
tf.Tensor(0.0476704322219276, shape=(), dtype=float64)
tf.Tensor(0.04677125095675787, shape=(), dtype=float64)
tf.Tensor(0.04347198868561798, shape=(), dtype=float64)
tf.Tensor(0.040349888999874975, shape=(), dtype=float64)
tf.Tensor(0.039206567552562105, shape=(), dtype=float64)
tf.Tensor(0.03883844075171264, shape=(), dtype=float64)
tf.Tensor(0.03761709111795794, shape=(), dtype=float64)
tf.Tensor(0.035421451452319806, shape=(), dtype=float64)
tf.Tensor(0.033151894515056374, shape=(), dtype=float64)
tf.Tensor(0.0315792089543398, shape=(), dtype=float64)
tf.Tensor(0.030718546749759115, shape=(), dtype=float64)
tf.Tensor(0.030000878763116478, shape=(), dtype=float64)
tf.Tensor(0.028806708016914304, shape=(), dt

In [7]:
print(state_fidelity(choi_target, channel_model.choi(T))/16)
print(state_fidelity(choi_target, channel_noisy.choi(T))/16)

tf.Tensor(0.9999997615504477, shape=(), dtype=float64)
tf.Tensor(0.9946733357721435, shape=(), dtype=float64)


In [13]:
J = channel_noisy.liouvillian.JumpOperator([1])
print(tf.linalg.trace(J))

tf.Tensor([-1.45756-1.23835j], shape=(1,), dtype=complex128)


In [14]:
print(tf.linalg.trace(tf.matmul(J, J, adjoint_a=True)))

tf.Tensor([1.+0.j], shape=(1,), dtype=complex128)


In [10]:
[print(param.numpy()) for param in channel_noisy.parameter_list]
[print(param.numpy()) for param in channel_model.parameter_list]

[ 0.297   -1.92892  1.08129]
[-0.48819 -1.75582  1.13198 -1.00127  2.37567 -0.37211 -0.04662  3.59378
  0.0816  -2.58003 -0.77131  1.28269  0.48629  1.05653  0.77224 -1.44921
 -0.90759 -0.53283]
[ 1.22099  0.16954  0.20735 -0.08243 -1.64848  0.82009 -0.6242   1.25484
 -1.21203 -2.48036 -2.30652 -0.78061]
[ 0.34881 -1.86274  0.86795]
[-0.64485 -1.71013  1.14861 -1.04768  2.03444  0.17889 -0.12125  3.20652
  0.29209 -2.50918 -0.58797  1.44163  0.94844  0.94947  0.80886 -1.46279
 -1.08694 -0.49586]
[ 1.15438  0.25876 -0.04822  0.03945 -1.3302   0.9655  -0.79513  0.97593
 -1.2466  -2.10461 -2.10313 -0.77807]


[None, None, None]

In [2]:
T = 1
U = tf.convert_to_tensor([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 0, 1],
                            [0, 0, 1, 0]], dtype=tf.complex128)

channel_target = ChoiMapStatic(U, mode="unitary")
choi_target = channel_target.choi
n = 2
d = 2**n

tf.random.set_seed(42)
np.random.seed(42)

H_model = SpinSpin(degree=3)
jump_operator = JumpOperator(4, 1)

lindblad_model = LindbladGenerator(hamiltonian = H_model, 
                                   jump_operator = jump_operator,
                                   gamma = 0.02)

channel_model = MagnusPropagator(liouvillian=lindblad_model, grid_size=200)
print(tf.math.reduce_sum(tf.abs(channel_model.choi(T) - choi_target)**2))

InvalidArgumentError: Input to reshape is a tensor with 50944 values, but the requested shape has 3184 [Op:Reshape]

In [ ]:
T = 1

optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_noisy.parameter_list)
        choi_model = channel_noisy.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_noisy.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_noisy.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_noisy.parameter_list))

In [34]:
T = 1

optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_model.parameter_list)
        choi_model = channel_model.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_model.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_model.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_model.parameter_list))

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(0.39437022669738675, shape=(), dtype=float64)
tf.Tensor(0.400409571409858, shape=(), dtype=float64)
tf.Tensor(0.3977104323397545, shape=(), dtype=float64)
tf.Tensor(0.3985157477176122, shape=(), dtype=float64)
tf.Tensor(0.39503818264711377, shape=(), dtype=float64)
tf.Tensor(0.3956585036527428, shape=(), dtype=float64)
tf.Tensor(0.3966427434598062, shape=(), dtype=float64)
tf.Tensor(0.39547078887813886, shape=(), dtype=float64)
tf.Tensor(0.39444261477352216, shape=(), dtype=float64)
tf.Tensor(0.3947386532895133, shape=(), dtype=float64)
tf.Tensor(0.39498455841510083, shape=(), dtype=float64)
tf.Tensor(0.3945293947584406, shape=(), dtype=float64)
tf.Tensor(0.39393631827258535, shape=(), dtype=float64)
tf.Tensor(0.39383976047731, shape=(), dtype=float64)
tf.Tensor(0.3940872450025843, shape=(), dtype=float64)
tf.Tensor(0.39411711741595445, shape=(), dtype=float64)
tf.Tensor(0.3937433426203658, shape=(), dtype=float64)
tf.Tensor(0.3933643447615358, shape=(), dtype=float64)
tf.Ten

In [35]:
print(state_fidelity(choi_target, channel_model.choi(T))/16)
print(state_fidelity(choi_target, channel_noisy.choi(T))/16)

tf.Tensor(0.8674173102190313, shape=(), dtype=float64)
tf.Tensor(0.8675157375294941, shape=(), dtype=float64)


## Fit HH

In [42]:
T = 1
#Hadamard gate 
H = tf.convert_to_tensor([[1, 1], [1, -1]], dtype=tf.complex128)/np.sqrt(2)
HH = kron(H,H)
 

channel_target = ChoiMapStatic(HH, mode="unitary")
choi_target = channel_target.choi
n = 2
d = 2**n

tf.random.set_seed(42)
np.random.seed(42)

H_model = SpinSpin(degree=2)
jump_operator = JumpOperator(4)

lindblad_model = LindbladGenerator(hamiltonian = H_model, 
                                   jump_operator = jump_operator,
                                   gamma = 0.0)

channel_model = MagnusPropagator(liouvillian=lindblad_model, grid_size=200)
print(tf.math.reduce_sum(tf.abs(channel_model.choi(T) - choi_target)**2))

tf.Tensor(31.939969369729063, shape=(), dtype=float64)


In [43]:
T = 1

optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(H_model.parameter_list)
        choi_model = channel_model.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_model.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, H_model.parameter_list)
    optimizer.apply_gradients(zip(gradients, H_model.parameter_list))

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(31.939969369729063, shape=(), dtype=float64)
tf.Tensor(31.921141384844432, shape=(), dtype=float64)
tf.Tensor(31.898446298611315, shape=(), dtype=float64)
tf.Tensor(31.87130703228202, shape=(), dtype=float64)
tf.Tensor(31.839340081737063, shape=(), dtype=float64)
tf.Tensor(31.802137319551917, shape=(), dtype=float64)
tf.Tensor(31.759172102791343, shape=(), dtype=float64)
tf.Tensor(31.709853404779906, shape=(), dtype=float64)
tf.Tensor(31.653633941259194, shape=(), dtype=float64)
tf.Tensor(31.58998260845262, shape=(), dtype=float64)
tf.Tensor(31.518339704394556, shape=(), dtype=float64)
tf.Tensor(31.43807487882883, shape=(), dtype=float64)
tf.Tensor(31.348450303740876, shape=(), dtype=float64)
tf.Tensor(31.248612816266764, shape=(), dtype=float64)
tf.Tensor(31.137591317140394, shape=(), dtype=float64)
tf.Tensor(31.014305861685884, shape=(), dtype=float64)
tf.Tensor(30.877658498992773, shape=(), dtype=float64)
tf.Tensor(30.726606811601012, shape=(), dtype=float64)
tf.Tensor(30.

In [44]:
channel_noisy = deepcopy(channel_model)
channel_noisy.liouvillian.gamma = 0.02
print(state_fidelity(choi_target, channel_model.choi(T))/16)
print(state_fidelity(choi_target, channel_noisy.choi(T))/16)

tf.Tensor(1.000000028827979, shape=(), dtype=float64)
tf.Tensor(0.8657969407036481, shape=(), dtype=float64)


In [45]:
T = 0.1

for j in tqdm(range(10)):
    optimizer = tf.optimizers.Adam(learning_rate=0.01)
    for i in range(200):
        with tf.GradientTape() as tape:
            tape.watch(channel_noisy.parameter_list)
            choi_noisy = channel_noisy.choi(T)
            choi_target1 = channel_model.choi(T)
            loss = tf.math.reduce_sum(tf.abs(choi_noisy - choi_target1) ** 2)

        print(loss)
        gradients = tape.gradient(loss, channel_noisy.parameter_list)
        optimizer.apply_gradients(zip(gradients, channel_noisy.parameter_list))
    T += 0.1

  0%|          | 0/10 [00:00<?, ?it/s]

tf.Tensor(0.009147243444276983, shape=(), dtype=float64)
tf.Tensor(0.009126555577356026, shape=(), dtype=float64)
tf.Tensor(0.008773020732179882, shape=(), dtype=float64)
tf.Tensor(0.008876395664071286, shape=(), dtype=float64)
tf.Tensor(0.009036115958726797, shape=(), dtype=float64)
tf.Tensor(0.008918247514213117, shape=(), dtype=float64)
tf.Tensor(0.008712404008023625, shape=(), dtype=float64)
tf.Tensor(0.008654891271613797, shape=(), dtype=float64)
tf.Tensor(0.008755013249368756, shape=(), dtype=float64)
tf.Tensor(0.008837212139936384, shape=(), dtype=float64)
tf.Tensor(0.008803064723578571, shape=(), dtype=float64)
tf.Tensor(0.008713519585043416, shape=(), dtype=float64)
tf.Tensor(0.00866645949781569, shape=(), dtype=float64)
tf.Tensor(0.008685694057977287, shape=(), dtype=float64)
tf.Tensor(0.00871895415106569, shape=(), dtype=float64)
tf.Tensor(0.008719506987036694, shape=(), dtype=float64)
tf.Tensor(0.008693938114919152, shape=(), dtype=float64)
tf.Tensor(0.008674097074374843, s

In [46]:
print(state_fidelity(choi_target, channel_model.choi(1))/16)
print(state_fidelity(choi_target, channel_noisy.choi(1))/16)

tf.Tensor(1.000000028827979, shape=(), dtype=float64)
tf.Tensor(0.8667670220121729, shape=(), dtype=float64)
